# Assignment 1 — Linear Regression with Polynomial Features & Ridge (Diabetes)
*Prepared:* 2025-10-11

**Goal:** Predict diabetes progression; compare plain Linear Regression vs PolynomialFeatures+Ridge; include a simple bootstrap CI for RMSE.

**Datasets:** `sklearn.datasets.load_diabetes()`

**Tools:** pandas, matplotlib, scikit-learn

In [ ]:
# Setup
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
plt.rcParams['figure.figsize'] = (7,4)

In [ ]:
# Load dataset into a DataFrame
data = load_diabetes()
X, y = data.data, data.target
df = pd.DataFrame(X, columns=data.feature_names).assign(target=y)
display(df.head())
display(df.describe())

In [ ]:
# Correlation heatmap (matplotlib imshow)
corr = df.corr(numeric_only=True)
fig, ax = plt.subplots()
im = ax.imshow(corr, interpolation='nearest')
ax.set_title('Correlation Heatmap')
ax.set_xticks(range(len(corr.columns))); ax.set_xticklabels(corr.columns, rotation=90)
ax.set_yticks(range(len(corr.index))); ax.set_yticklabels(corr.index)
fig.colorbar(im, ax=ax); plt.tight_layout(); plt.show()

In [ ]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

In [ ]:
# Pipeline A: StandardScaler -> LinearRegression
pipe_lr = Pipeline([('scaler', StandardScaler()),
                    ('lr', LinearRegression())])
pipe_lr.fit(X_train, y_train)

# Pipeline B: StandardScaler -> PolynomialFeatures(deg=2) -> Ridge
pipe_pr = Pipeline([('scaler', StandardScaler()),
                    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
                    ('ridge', Ridge(alpha=1.0, random_state=RANDOM_STATE))])
pipe_pr.fit(X_train, y_train)

def eval_model(model, Xtr, ytr, Xte, yte):
    pred_tr = model.predict(Xtr); pred_te = model.predict(Xte)
    res = {
        'MAE_tr': mean_absolute_error(ytr, pred_tr),
        'RMSE_tr': mean_squared_error(ytr, pred_tr, squared=False),
        'R2_tr': r2_score(ytr, pred_tr),
        'MAE_te': mean_absolute_error(yte, pred_te),
        'RMSE_te': mean_squared_error(yte, pred_te, squared=False),
        'R2_te': r2_score(yte, pred_te),
    }
    return res

res_lr = eval_model(pipe_lr, X_train, y_train, X_test, y_test)
res_pr = eval_model(pipe_pr, X_train, y_train, X_test, y_test)
pd.DataFrame([res_lr, res_pr], index=['Linear','Poly+Ridge'])

In [ ]:
# Residual plots
def residual_plot(model, X, y, title):
    pred = model.predict(X)
    resid = y - pred
    plt.scatter(pred, resid, s=18)
    plt.axhline(0, linestyle='--')
    plt.xlabel('Predicted'); plt.ylabel('Residual')
    plt.title(title); plt.show()

residual_plot(pipe_lr, X_test, y_test, 'Residuals — Linear Regression (test)')
residual_plot(pipe_pr, X_test, y_test, 'Residuals — Poly+Ridge (test)')

In [ ]:
# Bootstrap CI for RMSE (simple)
def bootstrap_rmse(model, X, y, B=200, random_state=RANDOM_STATE):
    rng = np.random.default_rng(random_state)
    rmses = []
    n = len(y)
    for _ in range(B):
        idx = rng.integers(0, n, size=n)
        yb = y[idx]; Xb = X[idx]
        pred = model.predict(Xb)
        rmses.append(mean_squared_error(yb, pred, squared=False))
    lower, upper = np.percentile(rmses, [2.5, 97.5])
    return np.mean(rmses), (lower, upper)

mean_rmse_lr, ci_lr = bootstrap_rmse(pipe_lr, X_test, y_test)
mean_rmse_pr, ci_pr = bootstrap_rmse(pipe_pr, X_test, y_test)
print('Linear RMSE bootstrap mean & 95% CI:', mean_rmse_lr, ci_lr)
print('Poly+Ridge RMSE bootstrap mean & 95% CI:', mean_rmse_pr, ci_pr)

**TODOs:**
- Add a scatter with best single feature vs target + fitted line.
- Write 5–8 bullet insights comparing both models and CI results.